In [16]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from bs4 import BeautifulSoup
import os

from selenium.webdriver.chrome.options import Options
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager,rc
from datetime import datetime
import time
import pandas as pd
import numpy as np
import cx_Oracle
import re
from tqdm import tqdm
from IPython import display

import urllib

In [17]:
naver_df = pd.read_csv('제주도_렌터카업체위치 (1).csv', encoding='cp949')
naver_df

,업체명,사업장구분,소재지도로명주소,소재지지번주소,위도,경도,차고지도로명주소,차고지지번주소,보유차고지수용능력,자동차총보유대수,...,평일운영종료시각,주말운영시작시각,주말운영종료시각,공휴일운영시작시각,공휴일운영종료시각,휴무일,홈페이지주소,대표자명,전화번호,데이터기준일자
0,㈜에스카,주사업장,제주특별자치도 제주시 연미길 42,제주특별자치도 제주시 오라삼동 2854-2,33.490170,126.505583,제주특별자치도 제주시 연미길 42,NaN,NaN,227,...,20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,070-4352-5515,2022-12-01
1,㈜인터파크렌터카,주사업장,제주특별자치도 제주시 용문로 88,제주특별자치도 제주시 용담이동 2739,33.503443,126.505186,제주특별자치도 제주시 용문로 88,NaN,NaN,212,...,20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,064-743-1515,2022-12-01
2,㈜아리랑,주사업장,제주특별자치도 제주시 용마서길 34,제주특별자치도 제주시 용담삼동 970-3,33.514957,126.502067,제주특별자치도 제주시 용담3동 969-3,NaN,NaN,534,...,20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,064-749-2139,2022-12-01
3,㈜제주아산렌트카,주사업장,제주특별자치도 제주시 연미길 44,제주특별자치도 제주시 오라삼동 2853-1,33.489788,126.505795,제주특별자치도 제주시 오라3동 2853-1,NaN,NaN,173,...,20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,064-743-9991,2022-12-01
4,한국렌트카㈜,주사업장,제주특별자치도 제주시 연미길 53,제주특별자치도 제주시 오라이동 2826-1,33.488469,126.505530,제주특별자치도 제주시 연미길 53,NaN,NaN,238,...,20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,064-748-5005,2022-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,㈜레드캡,영업소,제주특별자치도 제주시 다호길 130,제주특별자치도 제주시 용담이동 1963-4,33.503202,126.493994,제주특별자치도 제주시 연동 2456,NaN,NaN,620,...,20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,064-803-0391,2022-12-01
109,㈜제주스타렌탈,영업소,제주특별자치도 제주시 용문로 20,제주특별자치도 제주시 용담이동 1524-12,33.504892,126.498397,제주특별자치도 제주시 용담2동 1524-12,NaN,NaN,100,...,20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,064-759-3372,2022-12-01
110,㈜제주제일렌터카,영업소,제주특별자치도 제주시 월성로4길 78,제주특별자치도 제주시 용담이동 1730-4,33.498428,126.498207,제주특별자치도 제주시 애월읍 상귀리 679-1,NaN,NaN,170,...,20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,064-747-5670,2022-12-01
111,㈜해피네트웍스,영업소,제주특별자치도 제주시 일주서로 7581,제주특별자치도 제주시 이호이동 1606,33.495760,126.456506,제주특별자치도 제주시 노형동 3744-4,NaN,NaN,414,...,20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,064-711-7935,2022-12-01


In [18]:
naver_df = naver_df[['업체명', '소재지도로명주소', '홈페이지주소']]
naver_df.head()

,업체명,소재지도로명주소,홈페이지주소
0,㈜에스카,제주특별자치도 제주시 연미길 42,NaN
1,㈜인터파크렌터카,제주특별자치도 제주시 용문로 88,NaN
2,㈜아리랑,제주특별자치도 제주시 용마서길 34,NaN
3,㈜제주아산렌트카,제주특별자치도 제주시 연미길 44,NaN
4,한국렌트카㈜,제주특별자치도 제주시 연미길 53,NaN


In [19]:
naver_df['naver_map_url'] = ''

naver_df['naver_keyword'] = '제주도%20' + naver_df['업체명'] + '%20렌터카'
naver_df.head()

C:\Users\tj\AppData\Local\Temp\ipykernel_16764\3484429666.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  naver_df['naver_map_url'] = ''
C:\Users\tj\AppData\Local\Temp\ipykernel_16764\3484429666.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  naver_df['naver_keyword'] = '제주도%20' + naver_df['업체명'] + '%20렌터카'


,업체명,소재지도로명주소,홈페이지주소,naver_map_url,naver_keyword
0,㈜에스카,제주특별자치도 제주시 연미길 42,NaN,,제주도%20㈜에스카%20렌터카
1,㈜인터파크렌터카,제주특별자치도 제주시 용문로 88,NaN,,제주도%20㈜인터파크렌터카%20렌터카
2,㈜아리랑,제주특별자치도 제주시 용마서길 34,NaN,,제주도%20㈜아리랑%20렌터카
3,㈜제주아산렌트카,제주특별자치도 제주시 연미길 44,NaN,,제주도%20㈜제주아산렌트카%20렌터카
4,한국렌트카㈜,제주특별자치도 제주시 연미길 53,NaN,,제주도%20한국렌트카㈜%20렌터카


In [20]:
list=[]
for i in range(len(naver_df)):
    link='https://map.naver.com/v5/search/'
    link+=naver_df['naver_keyword'][i]
    list.append(link)
    
list

['https://map.naver.com/v5/search/제주도%20㈜에스카%20렌터카',
 'https://map.naver.com/v5/search/제주도%20㈜인터파크렌터카%20렌터카',
 'https://map.naver.com/v5/search/제주도%20㈜아리랑%20렌터카',
 'https://map.naver.com/v5/search/제주도%20㈜제주아산렌트카%20렌터카',
 'https://map.naver.com/v5/search/제주도%20한국렌트카㈜%20렌터카',
 'https://map.naver.com/v5/search/제주도%20㈜탐라렌트카%20렌터카',
 'https://map.naver.com/v5/search/제주도%20㈜써니빌렌트카%20렌터카',
 'https://map.naver.com/v5/search/제주도%20㈜용두암렌트카%20렌터카',
 'https://map.naver.com/v5/search/제주도%20㈜장수렌트카%20렌터카',
 'https://map.naver.com/v5/search/제주도%20㈜가자제주렌트카%20렌터카',
 'https://map.naver.com/v5/search/제주도%20오름렌트카㈜%20렌터카',
 'https://map.naver.com/v5/search/제주도%20제주렌트카㈜%20렌터카',
 'https://map.naver.com/v5/search/제주도%20㈜제주오케이렌터카%20렌터카',
 'https://map.naver.com/v5/search/제주도%20㈜뉴제주렌터카%20렌터카',
 'https://map.naver.com/v5/search/제주도%20㈜굿모닝렌트카%20렌터카',
 'https://map.naver.com/v5/search/제주도%20(유)제주공항렌트카%20렌터카',
 'https://map.naver.com/v5/search/제주도%20메트로렌트카㈜%20렌터카',
 'https://map.naver.com/v5/search/제주도%20㈜한길렌터카%20렌터

In [42]:
naver_df['naver_map_url'] = list
naver_df

C:\Users\tj\AppData\Local\Temp\ipykernel_16764\3150178210.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  naver_df['naver_map_url'] = list


,업체명,소재지도로명주소,홈페이지주소,naver_map_url,naver_keyword
0,㈜에스카,제주특별자치도 제주시 연미길 42,NaN,https://map.naver.com/v5/search/제주도%20㈜에스카%20렌터카,제주도%20㈜에스카%20렌터카
1,㈜인터파크렌터카,제주특별자치도 제주시 용문로 88,NaN,https://map.naver.com/v5/search/제주도%20㈜인터파크렌터카...,제주도%20㈜인터파크렌터카%20렌터카
2,㈜아리랑,제주특별자치도 제주시 용마서길 34,NaN,https://map.naver.com/v5/search/제주도%20㈜아리랑%20렌터카,제주도%20㈜아리랑%20렌터카
3,㈜제주아산렌트카,제주특별자치도 제주시 연미길 44,NaN,https://map.naver.com/v5/search/제주도%20㈜제주아산렌트카...,제주도%20㈜제주아산렌트카%20렌터카
4,한국렌트카㈜,제주특별자치도 제주시 연미길 53,NaN,https://map.naver.com/v5/search/제주도%20한국렌트카㈜%2...,제주도%20한국렌트카㈜%20렌터카
...,...,...,...,...,...
108,㈜레드캡,제주특별자치도 제주시 다호길 130,NaN,https://map.naver.com/v5/search/제주도%20㈜레드캡%20렌터카,제주도%20㈜레드캡%20렌터카
109,㈜제주스타렌탈,제주특별자치도 제주시 용문로 20,NaN,https://map.naver.com/v5/search/제주도%20㈜제주스타렌탈%...,제주도%20㈜제주스타렌탈%20렌터카
110,㈜제주제일렌터카,제주특별자치도 제주시 월성로4길 78,NaN,https://map.naver.com/v5/search/제주도%20㈜제주제일렌터카...,제주도%20㈜제주제일렌터카%20렌터카
111,㈜해피네트웍스,제주특별자치도 제주시 일주서로 7581,NaN,https://map.naver.com/v5/search/제주도%20㈜해피네트웍스%...,제주도%20㈜해피네트웍스%20렌터카


In [46]:
options = Options()
options.add_experimental_option("detach", True)
service = Service(ChromeDriverManager().install())
naver_driver = webdriver.Chrome(service=service, options=options)

In [47]:
all_data_lst = []


for i, keyword in enumerate(naver_df['naver_keyword'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {naver_df.shape[0] -1} 행", keyword)
    a_list = []
    name_lst = []
    star_lst = []
    oper_list = []
    nic_list = []
    re_list = []
    date_list = []
    
    try:
        naver_map_url = f"https://map.naver.com/v5/search/{keyword}"

        naver_driver.get(naver_map_url)
        naver_driver.implicitly_wait(5)
        
        naver_driver.switch_to.frame('entryIframe')
        naver_driver.implicitly_wait(5)
        #==================================    
        # 가게이름
        try:
            naver_driver.find_element(By.CLASS_NAME,'Fc1rA')
            naver_driver.implicitly_wait(5)
            name = naver_driver.find_element(By.CLASS_NAME,'Fc1rA').text
            naver_driver.implicitly_wait(5)
            print(name)
            name_lst.append(name)
            
        except:
            print('가게없음')
            name_lst.append('')
        
        # 별점 수집
        try:
            naver_driver.find_element(By.CLASS_NAME,'PXMot.LXIwF')
            naver_driver.implicitly_wait(5)
            star = naver_driver.find_element(By.CLASS_NAME,'PXMot.LXIwF').text
            naver_driver.implicitly_wait(5)
            print(star)
            star_lst.append(star)
            
        except:
            print('별점없음')
            star_lst.append('')
        
        # 운영시간 수집
        try:
            naver_driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div/div[2]/div/ul/li[2]/div/a')
            naver_driver.implicitly_wait(5)
            naver_driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div/div[2]/div/ul/li[2]/div/a').click()
            naver_driver.implicitly_wait(5)
            oper = naver_driver.find_element(By.CLASS_NAME,'kGc0c').text
            print(oper)
            oper_time = naver_driver.find_element(By.CLASS_NAME,'qo7A2').text
            print(oper_time)
            oper_list.append(oper+' '+oper_time)
            
        except:
            print('운영시간없음')
            oper_list.append('')
        
        
        # 리뷰 수집
        try:
            naver_driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[2]')
            naver_driver.implicitly_wait(5)
            naver_driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[2]').click()
            naver_driver.implicitly_wait(5)
            print('리뷰 클릭')
            
            
            try:
                scroll_location = naver_driver.execute_script("return document.body.scrollHeight")

                while True:
                    naver_driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

                    time.sleep(2)

                    scroll_height = naver_driver.execute_script("return document.body.scrollHeight")


                    if scroll_location == scroll_height:
                        break
                    else:
                        scroll_location = naver_driver.execute_script("return document.body.scrollHeight")
                    
                    naver_driver.implicitly_wait(5)
                    #print('스크롤 내림')

            except:
                print('스크롤 안됨')
            
            # 더보기 클릭
            try:
                naver_driver.find_element(By.CLASS_NAME,'fvwqf')
                naver_driver.implicitly_wait(5)
                
                while True:
                    naver_driver.find_element(By.CLASS_NAME,'fvwqf').click()
                    naver_driver.implicitly_wait(5)
                    #print('더보기 클릭')
                    
                    try:
                        scroll_location = naver_driver.execute_script("return document.body.scrollHeight")

                        while True:
                            naver_driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")

                            time.sleep(2)

                            scroll_height = naver_driver.execute_script("return document.body.scrollHeight")

                            if scroll_location == scroll_height:
                                break

                            else:
                                scroll_location = naver_driver.execute_script("return document.body.scrollHeight")

                            naver_driver.implicitly_wait(5)
                            #print('스크롤 내림')

                    except:
                        print('스크롤 안됨')
                        
                    if not naver_driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[5]/div/div/div/div/a[2]'):
                        break

            except:
                print('더보기 안됨')

            try:
                naver_driver.find_element(By.CLASS_NAME,'sBWyy')
                naver_driver.implicitly_wait(5)
                print('sBWyy 있음')
                
                naver_html = naver_driver.page_source
                naver_soup = BeautifulSoup(naver_html, 'lxml')
                naver_driver.implicitly_wait(5)
                print('페이지 소스 가져옴')

                for nic in naver_soup.find_all('div', attrs = {'class':'sBWyy'}):
                    #print(nic.text)
                    nic_list.append(nic.text)
                
                
                try:
                    tags = naver_soup.find_all('span', {"class" : "YeINN"})  
#                     rent_coment=re.compile('zPfVt["][>](.*)[<][/]span[>][<][/]a')
#                     review=rent_coment.findall(str(tags))
                    naver_driver.implicitly_wait(3)
#                    if naver_soup.find_all('span', attrs = {'class':'zPfVt'}):
                    if 'zPfVt' in tags:
                        for review in tags:
                            rent_coment=re.compile('zPfVt["][>](.*)[<][/]span[>][<][/]a')
                            rent_review=rent_coment.findall(str(review))
                            print(revrent_reviewiew)
                            re_list.append(rent_review)
                            
                    else:
                        print('글 리뷰없음')
                        re_list.append('')
                except:
                    print('글 리뷰없음')
                    re_list.append('')
                    
                try:
                    naver_driver.implicitly_wait(3)   
                    for re_date in naver_soup.find_all('time', attrs = {'aria-hidden':'true'}):
                        #print(re_date.text)
                        date_list.append(re_date.text)
                except:
                    print('리뷰 날짜 없음')
                    date_list.append('')
                    
            except:
                print('닉네임 없음')
                nic_list.append('')
            
        except:
            print('리뷰 클릭 안됨')
            nic_list.append('')
            re_list.append('')
            date_list.append('')
#             tag_list.append('')
        
            
        try:
            is_store_exist= naver_driver.find_element(By.CLASS_NAME,'FYvSc')

            is_store_exist.text=='조건에 맞는 업체가 없습니다.'
            a='x'
            print('x')
            a_list.append(a)
        except:
            pass

        print('\n'*3)
            
        
    except Exception as e1:
        a='o'
        print(e1)
        a_list.append(a)
        
        if "li:nth-child(1)" in str(e1):
            try:
                naver_df.iloc[i,-2] = naver_driver.find_element(By.CLASS_NAME,"XUrfU")

            except Exception as e2:
                print(e2)
                naver_df.iloc[i,-2] = naver_driver.find_element(By.CLASS_NAME,"XUrfU")
                time.sleep(1)
        else:
            pass
    
    for row in range(len(nic_list)):
        print(f'지금{row}번째')
        print(len(nic_list))
        print(len(re_list))
        print(len(date_list))
        row_list = []
        row_list.append(name_lst[0])
        row_list.append(star_lst[0])
        row_list.append(oper_list[0])
        #=============================
        row_list.append(nic_list[row])
        row_list.append(re_list[row])
        row_list.append(date_list[row])
        
        all_data_lst.append(row_list)
        


naver_driver.quit()

이번에 찾을 키워드 : 0 / 112 행 제주도%20㈜에스카%20렌터카
Message: entryIframe

이번에 찾을 키워드 : 1 / 112 행 제주도%20㈜인터파크렌터카%20렌터카
인터파크렌터카
별점
4.16/5
매일
08:00 - 20:00
리뷰 클릭
더보기 안됨
sBWyy 있음
페이지 소스 가져옴
글 리뷰없음




지금0번째
238
1
238
지금1번째
238
1
238


IndexError: list index out of range

In [24]:
all_data_lst

[['에스카', '', '', '다그런것', '친절해요', '22.12.30.금'],
 ['에스카', '', '', '2rm****', '굿', '21.10.26.화'],
 ['인터파크렌터카',
  '별점\n4.16/5',
  '매일 08:00 - 20:00',
  '양많은여성',
  '돌하루팡에서 연결시켜준 곳. 무상으로 업그레이드\n도 해주고, 엄청 친절은 아니지만 차 상태도 새 차처\n럼 깨끗하고 만족스럽다. 연료를 충전하지 않...',
  '1.9.월'],
 ['인터파크렌터카', '별점\n4.16/5', '매일 08:00 - 20:00', '깨비9372', '굿', '1.4.수'],
 ['인터파크렌터카',
  '별점\n4.16/5',
  '매일 08:00 - 20:00',
  '연두47',
  '첫 이용인데 생각보다 괜찮고 가격도 좋았어요\n(20 15)',
  '1.3.화'],
 ['인터파크렌터카', '별점\n4.16/5', '매일 08:00 - 20:00', 'lja****', '굿', '22.12.30.금'],
 ['인터파크렌터카', '별점\n4.16/5', '매일 08:00 - 20:00', 'lja****', '굿', '22.12.28.수'],
 ['인터파크렌터카',
  '별점\n4.16/5',
  '매일 08:00 - 20:00',
  'Inhyeggo',
  '그냥 편하게 운전 잘했어요\n결항으로 이틀연장했는데 직원분 친절하게 상담해주셨어요',
  '22.12.27.화'],
 ['인터파크렌터카',
  '별점\n4.16/5',
  '매일 08:00 - 20:00',
  'Inhyeggo',
  '굿 !!! 너무 좋아요',
  '22.12.25.일'],
 ['인터파크렌터카',
  '별점\n4.16/5',
  '매일 08:00 - 20:00',
  'Poohglet0',
  '좋아요!!',
  '22.12.23.금'],
 ['인터파크렌터카',
  '별점\n4.16/5',
  '매일 08:00 - 20:00',
  '워니거니91',
  '친절하고 

In [ ]:
naver_re_df = naver_df[['id','업체명']]

In [13]:
star_lst

['',
 '별점\n4.16/5',
 '',
 '별점\n4.32/5',
 '',
 '',
 '',
 '별점\n3.99/5',
 '별점\n4.44/5',
 '별점\n4.27/5',
 '',
 '',
 '',
 '별점\n3.84/5',
 '별점\n4.02/5',
 '',
 '별점\n3.68/5',
 '별점\n4.04/5',
 '',
 '별점\n3.61/5',
 '',
 '',
 '',
 '별점\n3.86/5',
 '',
 '별점\n3.92/5',
 '',
 '별점\n4.03/5',
 '',
 '',
 '',
 '',
 '별점\n3.77/5',
 '별점\n4.09/5',
 '',
 '',
 '',
 '',
 '별점\n3.84/5',
 '',
 '',
 '별점\n4.08/5',
 '별점\n4/5',
 '',
 '별점\n3.75/5',
 '',
 '',
 '',
 '',
 '별점\n4.11/5',
 '',
 '',
 '',
 '별점\n4.19/5',
 '',
 '별점\n4.2/5',
 '별점\n3.82/5',
 '별점\n3.96/5',
 '',
 '별점\n4.2/5',
 '',
 '별점\n4.16/5',
 '',
 '별점\n3.87/5',
 '',
 '',
 '',
 '별점\n4.44/5',
 '',
 '별점\n4.11/5',
 '',
 '별점\n3.91/5',
 '',
 '별점\n3.98/5',
 '',
 '',
 '별점\n4.17/5',
 '별점\n4.14/5',
 '',
 '별점\n4.28/5',
 '',
 '별점\n4.05/5',
 '',
 '별점\n4.36/5',
 '',
 '',
 '',
 '',
 '별점\n3.25/5',
 '별점\n4.12/5',
 '',
 '별점\n4.01/5',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '별점\n4.46/5',
 '',
 '별점\n4.37/5',
 '',
 '',
 '',
 '별점\n3.54/5']

In [50]:
a_list

['o', 'o', 'o']

In [14]:
len(star_lst)

113

In [15]:
naver_df_2 = pd.DataFrame(star_lst)
naver_df_2

,0
0,
1,별점\n4.16/5
2,
3,별점\n4.32/5
4,
...,...
108,별점\n4.37/5
109,
110,
111,


In [18]:
naver_df_2.rename(columns={0:'star'}, inplace=True)
naver_df_2

,star
0,
1,별점\n4.16/5
2,
3,별점\n4.32/5
4,
...,...
108,별점\n4.37/5
109,
110,
111,


In [20]:
naver_df_2['star'] = naver_df_2['star'].str.replace('별점\n', '')
naver_df_2

,star
0,
1,4.16/5
2,
3,4.32/5
4,
...,...
108,4.37/5
109,
110,
111,


In [21]:
naver_df_2['star'] = naver_df_2['star'].str.replace('/5', '')
naver_df_2

,star
0,
1,4.16
2,
3,4.32
4,
...,...
108,4.37
109,
110,
111,


In [23]:
naver_df_2.to_csv('naver_star_1.csv', encoding = 'cp949')

In [29]:
naver_map_url = 'https://map.naver.com/v5/search/%EC%A0%9C%EC%A3%BC%EB%8F%84%20%EC%95%84%EB%A6%AC%EB%9E%91?c=14112466.6801357,3948881.0670945,15,0,0,0,dh'
naver_driver.get(naver_map_url)
naver_driver.switch_to.frame('searchIframe')
naver_driver.implicitly_wait(3)

In [110]:
star = naver_driver.find_element(By.CLASS_NAME,'PXMot.LXIwF').text
star

'별점\n4.16/5'

In [51]:
naver_star = re.findall('(\d*\.?\d+)[/]\d', str(star))
print(naver_star)

['4.16']


In [53]:
naver_star = re.compile('(\d*\.?\d+)[/]\d').search(star)
print(naver_star)

<re.Match object; span=(3, 9), match='4.16/5'>


In [66]:
star_lst

[['4.16']]

In [32]:
naver_driver.find_element(By.CLASS_NAME,'YzBgS').text == '렌터카'

False